<a href="https://colab.research.google.com/github/rakaputra12/Data_Exploration_Diabetes/blob/main/DEP_New_neu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Dependencies

#!pip install pandas
#!pip install numpy
#!pip install scikit-learn
#!pip install matplotlib
#!pip install mlflow
#!pip install seaborn
#!pip install mlxtend
#!pip install graphviz

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import accuracy_score


%matplotlib inline

# **Auswahl Datensatz**

Hier wurde der Datensatz "Diabetes" ausgewählt.

In [ ]:
#Import Dataset
dataset = "https://raw.githubusercontent.com/rakaputra12/Data_Exploration_Diabetes/main/Healthcare-Diabetes.csv"
#dataset_local = "Data_Exploration_Diabetes/Healthcare-Diabetes.csv"
diabetes_df = pd.read_csv(dataset)

In [ ]:
diabetes_df.head()

In [ ]:
#Delete the column "Id"
diabetes_df = diabetes_df.drop('Id', axis=1)

# **Charakterisierung des Datensatzes**

In [ ]:
diabetes_df.columns

In [ ]:
diabetes_df.info()

In [ ]:
diabetes_df.describe()

In [ ]:
diabetes_df.describe().T

In [ ]:
diabetes_df.isnull()

In [ ]:
diabetes_df.isnull().sum()

In [ ]:
zero_counts = (diabetes_df == 0).sum()

In [ ]:
zero_counts

On these columns, a value of zero does not make sense and thus indicates missing value

Following columns or variables have an invalid zero value: Glucose, BloodPressure, SkinThickness, Insulin, BMI: it is better to replace zeros with NaN since after that counting them would be easier and zeros need to be replaced with suitable values. On the column like Pregnancies, a value of zero does make sense, cause someone can have never been pregnant. For the column "Outcome" is because the value for sufferer or not is only differentiated by one or zero.

In dem Fall kann es eigentlich nicht berücksichtigt werden. Dazu können aber negative Werte bei Heatmap liefern.

Oder hier kann ich einfach Zeile löschen, die Null Werte haben. so kann ich bei Metriks nicht so viel Gedanken machen.????

In [ ]:
#Replace zeros with NaN Value
diabetes_df_copy = diabetes_df.copy(deep = True)
diabetes_df_copy[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = diabetes_df_copy[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0,np.NaN)

#Showing the Count of NaNs
print(diabetes_df_copy.isnull().sum())

In [ ]:
#Aiming to impute NaN values for the columns in accordance with their distribution
diabetes_df_copy['Glucose'].fillna(diabetes_df_copy['Glucose'].mean(), inplace=True)
diabetes_df_copy['BloodPressure'].fillna(diabetes_df_copy['BloodPressure'].mean(), inplace=True)
diabetes_df_copy['SkinThickness'].fillna(diabetes_df_copy['SkinThickness'].mean(), inplace=True)
diabetes_df_copy['Insulin'].fillna(diabetes_df_copy['Insulin'].mean(), inplace=True)
diabetes_df_copy['BMI'].fillna(diabetes_df_copy['BMI'].mean(), inplace=True)

Nun ist der Datensatz bereit zu verarbeiten.

## **Perform EDA (Exploratory Data Analysis )**

In [ ]:
#Checking the balance of the data by plotting the count of outcomes by their values
color_wheel = {0: '#0392cf', 1: '#7bc043'}
colors = diabetes_df["Outcome"].map(lambda x: color_wheel.get(x))
print(diabetes_df.Outcome.value_counts())
p = diabetes_df.Outcome.value_counts().plot(kind="bar", color = colors)

The above graph shows that the data is biased towards datapoints having outcome value as 0 where it means that diabetes was not present actually. The number of non-diabetics is almost twice the number of diabetic patients.

Is my statement above correct?? i don't think so :( or correct??? 34% diabet, 66% no diabets

**HIER MUSS WIRKLICH BESCHRIEBEN WERDEN, DASS DAS EINEN EINFLUSS BEI DER GRAFIK BEI K NEIGHBOR HAT!!!!!!! ????????**

Wenn ich noch mit KKN arbeiten würde :)

In [ ]:
#Plotting the Pair Plots for the uncleaned data
p = sns.pairplot(diabetes_df, hue= 'Outcome')

In [ ]:
#Plotting the Pair Plots for the cleaned data
p = sns.pairplot(diabetes_df_copy, hue= 'Outcome')

## **Correlation between all the features**

In [ ]:
#Correlation between all the features before cleaning
plt.figure(figsize= (12,10))
p = sns.heatmap(diabetes_df.corr(), annot = True, cmap = 'RdYlGn')

In [ ]:
#Correlation between all the features after cleaning
plt.figure(figsize= (12,10))
p = sns.heatmap(diabetes_df_copy.corr(), annot = True, cmap = 'RdYlGn')

HIER KANN MAN ERKENNEN, DASS DIE KORRELATION BETWENN FEATURES SICH ERHÖHEN, NACHDEM DER DATENSATZ BEREINIGHT WORDEN IST. Z.B GLUCOSE UND OUTCOME

BEFORE : 0.46 AFTER : 0.49

# **Feature Engineering**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Hier kann ich auch mit Preprocessing data tun. also mit StandardScaler. See the previous Code in other Colab.

In [ ]:
X=diabetes_df_copy[diabetes_df_copy.columns[0:-1]]
y=diabetes_df_copy[diabetes_df_copy.columns[-1]]

## **Split the dataset**

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,random_state=42)

## **Ermittlung der Feature Importance mit einem Decision Tree**

In [ ]:
tree = DecisionTreeClassifier(max_depth=4,random_state=0)
tree.fit(X_train,y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train,y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test,y_test)))

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(tree,out_file="diabetes_tree.dot",class_names=["0","1"],
feature_names=X.columns,impurity=False,filled=True)
import graphviz
with open("diabetes_tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

In [ ]:
print("Feature importances:\n{}".format(tree.feature_importances_))

In [ ]:
df_feature_importance = pd.DataFrame({'Feature Names': X.columns, 'Importance of  Feature': tree.feature_importances_ })
df_feature_importance

In [ ]:
def plot_feature_importances_adult_census(model):
    n_features = X.shape[1]
    plt.barh(range(n_features),model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features),X.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    ##plt.show()
    ##fig=plt.figure()
    plt.savefig("feature_imporatnace_diabetes.png")
    plt.show()
    plt.close()
plot_feature_importances_adult_census(tree)

Hier kann man erkennen, dass Feature "Glucose" und "BMI" Parameter sind, auf die geachtet werden müssen.

## **Ermittlung der Feature Importance mit einem Random  (Zum Vergleich)**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest = RandomForestClassifier(n_estimators=100,random_state=0)
forest.fit(X_train, y_train)

In [ ]:
print(forest.feature_importances_)

In [ ]:
df_feature_importance_withRandomForest = pd.DataFrame({'Feature Names': X.columns, 'Importance of  Feature': forest.feature_importances_ })
df_feature_importance_withRandomForest

In [ ]:
def plot_feature_importances_adult_census(model):
    n_features = X.shape[1]
    plt.barh(range(n_features),model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features),X.columns)
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature")
    ##plt.show()
    ##fig=plt.figure()
    plt.savefig("feature_imporatnace_diabetes_forest.png")
    plt.show()
    plt.close()
plot_feature_importances_adult_census(forest)

# **Split des Datensatzes**

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,random_state=101, test_size=0.2)

nach unten verschoben????

# **Auswahl der Metriken**

When working with imbalanced datasets (which is often the case in medical datasets like diabetes prediction), metrics like precision, recall, and AUC-ROC become particularly important as they provide insights into the model's performance beyond simple accuracy.

Therefore, focusing on precision, recall, **F1 score**, and AUC-ROC will give you a more comprehensive understanding of your model's performance in this context.

If False Positives are Costly: Choose Precision. Precision measures the proportion of true positive predictions out of all positive predictions. If false positives are costly in your scenario (e.g., unnecessary treatments or interventions for individuals who don't have diabetes), then prioritize precision.

If False Negatives are Costly: Choose Recall. Recall measures the proportion of true positive predictions out of all actual positive instances. If missing positive cases (false negatives) is more critical in your context (e.g., failing to diagnose individuals who actually have diabetes), then prioritize recall.

If You Need a Balance: Choose the F1 Score. The F1 score is the harmonic mean of precision and recall, providing a balanced measure of both. It's useful when you want to balance the trade-off between false positives and false negatives.

So F1-Score is better???

# **Auswahl und Beschreibung der ML-Methode**

Methode that im gonna use is supervised learning.

A Support Vector Machine (SVM) is a supervised machine learning algorithm used for classification and regression tasks. SVM is particularly well-suited for classification of complex datasets where there is clear margin of separation between classes.


To have even better result we decided to go for pipelining and used the Support Vector Machine algorithm.

The best score and parameter that we found were 0.75??? using Linear Kernel. The testing accuracy achieved was of 80.5%.???

# **Implementierung Training**

In [ ]:
from sklearn.linear_model import SGDClassifier
ppn = SGDClassifier(loss='perceptron')
lr = SGDClassifier(loss='log')
svm = SGDClassifier(loss='hinge')

Overall, SGDClassifier is a versatile and efficient tool for training linear classifiers, particularly in situations where scalability and efficiency are essential, such as with large datasets or online learning scenarios.

Jetzt nehmen wir nur die zweibeste Features für die Training

In [ ]:
X = diabetes_df_copy[['Glucose','BMI']]
y = diabetes_df_copy['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
# To make Plotting

#X_train = np.array(X_train)
#y_train = np.array(y_train)
#X_test = np.array(X_test)
#y_test = np.array(y_test)

### **Kernel --> General**

In [ ]:
from sklearn import svm
machine1 = svm.SVC()
machine1.fit(X_train,y_train)
y_pred1 = machine1.predict(X_test)

In [ ]:
#plot_decision_regions(X_train, y_train, machine1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print(classification_report(y_test, y_pred1))

# **Hyperparametertuning Verwendung ML Lifecycle Mgt im Code**

In dem Fall kommt Ml flow zum Einsatz

In [ ]:
#!pip install mlflow
#!pip install pyngrok

In [ ]:
#!mlflow

In [ ]:
#import mlflow.sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
pipe_svc = Pipeline([('scl', StandardScaler()),('clf', SVC(random_state=1))])
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [{'clf__C': param_range,'clf__kernel': ['linear']},{'clf__C': param_range,'clf__gamma': param_range,'clf__kernel': ['rbf']}]
gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='accuracy',cv=10,n_jobs=-1)
gs = gs.fit(X_train, y_train)

kernel : poly rausschmeißen??? then dauert nur 17 sec

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
'''from sklearn.svm import SVC
# Build model using grid search and cross fold validation
parameters = {'C': [0.1, 1, 10, 100],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto']}
metrics = ['f1', 'recall', 'precision', 'roc_auc', 'neg_log_loss', 'neg_brier_score',
           'average_precision', 'balanced_accuracy']

svm = SVC(random_state=0,probability=True)
clf = GridSearchCV(svm, parameters, scoring=metrics, refit='f1')
clf.fit(X_train,y_train)

# Log artifacts to MLflow
mlflow.sklearn.log_model(clf.best_estimator_, "best model")
mlflow.log_metric('best score', clf.best_score_)
for k in clf.best_params_.keys():
    mlflow.log_param(k, clf.best_params_[k])
print("Model saved in run %s" % mlflow.active_run().info.run_uuid)'''

In [ ]:
'''from sklearn.svm import SVC

# Build model using randomized search and cross-fold validation
parameters = {'C': [0.1, 1, 10, 100],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto']}
metrics = ['f1', 'recall', 'precision', 'roc_auc', 'neg_log_loss', 'neg_brier_score',
           'average_precision', 'balanced_accuracy']

svm = SVC(random_state=0, probability=True)
clf = RandomizedSearchCV(svm, parameters, scoring=metrics, refit='f1', n_iter=10, random_state=0)
clf.fit(X_train, y_train)

# Log artifacts to MLflow
mlflow.sklearn.log_model(clf.best_estimator_, "best model")
mlflow.log_metric('best score', clf.best_score_)
for k in clf.best_params_.keys():
    mlflow.log_param(k, clf.best_params_[k])
print("Model saved in run %s" % mlflow.active_run().info.run_uuid)'''

In [ ]:
'''from pyngrok import ngrok

ngrok.kill()

NGROK_AUTH_TOKEN = "2eMIaE6nA3kl4wBKRC2l5fiXQqj_2pbU9j8dNWagQCRKNtMNq"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr=5000, proto="http", bind_tls= True)
print("Mflow Tracking UI:", ngrok_tunnel.public_url)'''

In [ ]:
'''!mlflow ui'''

# **Evaluation und Ergebnisdarstellung**

In [ ]:
y_pred = gs.predict(X_test)

print(classification_report(y_test, y_pred))

**DONE????**

# **Vorhersage-Demo**